In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# reading the CSV file into pandas dataframe
df = pd.read_csv("/kaggle/input/dl-intro-to-nn/car-mpg.csv")  
print(df.shape)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df  = df.drop("car_name" , axis=1)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# print(df['hp']).value_counts()

In [ ]:
# df.isna().sum()

In [ ]:
# temp  = pd.DataFrame(df.hp.str.isdigit()) 
# temp

In [ ]:
# temp.sum()

In [ ]:
# temp[temp['hp'] == False] 

In [ ]:
# On inspecting records number 32, 126 etc, we find "?" in the columns. Replace them with "nan"
#Replace them with nan and remove the records from the data frame that have "nan"
df = df.replace('?', np.nan)

In [ ]:
df.isna().sum()

In [ ]:
# Since the question marks appear only in the "hp" columns, replace them with the median of the same column
#df = df.apply(lambda x: x.fillna(x.median()),axis=0)
df.dropna(inplace=True)

In [ ]:
# df.isna().sum()

In [ ]:
df.shape

In [ ]:
# While reading the data into dataframe, Pandas assigned the data type as object to column "hp" because of the question marks
# The column type continues to be same even after replacing the question marks with the median
# Convert the column type to float64


df['hp'] = df['hp'].astype('float64')  # converting the hp column from object / string type to float

In [ ]:
df

In [ ]:
import seaborn as sns

sns.pairplot(data=df);

In [ ]:
# convert origin to numric values (but that will introduce an order in that column)

df['Origin'] = df['origin'].map(lambda x: {1: 'USA', 2: 'Europe', 3: 'Japan'}.get(x))

In [ ]:
# convert the numeric values to one-hot code to remove the order

df = pd.get_dummies(df, columns=['origin'],drop_first=True)

In [ ]:
df.head()

In [ ]:
df.drop(['Origin'],axis=1,inplace=True)
print(df.shape)
df.head()

In [ ]:
# Import `train_test_split` from `sklearn.model_selection`
from sklearn.model_selection import train_test_split

# Specify the data 
X=df.iloc[:,1:]

# Specify the target labels and flatten array
y= df.mpg


# Split the data up in train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_train =  np.array(y_train)
y_test =  np.array(y_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

In [ ]:
X_train.shape

# <center> Build Deep Neural Network

In [ ]:
# Using Tensorflow Keras instead of the original Keras

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


ncols = X_train.shape[1]

# Initialize the constructor
model = Sequential()

# Add an first hidden layer 
model.add(Dense(20, input_shape=(ncols,),activation='sigmoid'))

# second hidden layer
#model.add(Dense(10, activation='relu', kernel_initializer='normal'))


# Add an output layer with one neuron and no activation specified
model.add(Dense(1))

model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(loss='mean_squared_error',optimizer=optimizer , metrics = ['mae' , 'mse'])

In [ ]:
epochs = 50
history = model.fit(X_train, y_train, epochs=epochs, validation_split = 0.2, verbose = True)
hist  = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

In [ ]:
hist.head()

In [ ]:
model.summary()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hist['mse'])
plt.plot(hist['val_mse'])
plt.legend(("train" , "valid") , loc =0)

In [ ]:
plt.plot(hist['mae'])
plt.plot(hist['val_mae'])
plt.legend(("train" , "valid") , loc =0)

In [ ]:
loss = model.evaluate(X_test, y_test, verbose=False)

In [ ]:
y_pred = np.round(model.predict(X_test))

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, y_pred)